In [2]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/data/users/zliu/mend/notebooks'

In [27]:
task = "musique"
edit_loss="clm"
edit_input="question"
prompt = "urial"
exp_name = "llama3.2-1B_on_musique" # llama3.2-1B_on_musiqueQonly
fpath = f"../exp_output/{exp_name}/{task}/mend_eval_loss={edit_loss}_input={edit_input}_n=1000_prompt={prompt}.xlsx"
# print("File name:", fpath)
# fpath = "/u/zliu/datastor1/mend/exp_output/musique_injector/musique/mend_eval_loss=clm_input=question_n=1000_prompt=no_w-gen_wo-icl_textidx1_split.xlsx"
fpath = "/u/zliu/datastor1/mend/exp_output/Llama-3.2-1B-eos-sft_clm-baseline_input=seen-hop_lr=1e-05_epoch=4.0/all_table_new_e+s.xlsx"
do_gen = "w-gen" in fpath
df = pd.read_excel(fpath)
metrics = ["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"]
if do_gen:
    metrics += ["rouge1", "llm_accuracy"]
# macro_averaging(df, multi_levelz_averaging=["stage", "question"], metrics=['[Q][A] acc', "rouge1", "llm_accuracy"])
len(df)


9000

In [10]:
df = pd.read_excel("/data/users/zliu/mend/debug_exp_output/llama3.2-1B-eos-sft/common/base_n=100_prompt=no_w-gen_wo-icl_increment.xlsx")
df["is_num"] = df["is_num"].astype(float)
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff", "diff"]].round(2)

,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
count,100.00,100.00,100.00,100.0,100.00,100.00
mean,0.07,0.68,0.07,1.0,19.88,-16.34
std,0.26,0.15,0.26,0.0,73.46,74.33
min,0.00,0.50,0.00,1.0,0.00,-680.00
25%,0.00,0.50,0.00,1.0,1.00,-7.50
50%,0.00,0.75,0.00,1.0,1.00,0.00
75%,0.00,0.75,0.00,1.0,13.00,1.00
max,1.00,1.00,1.00,1.0,680.00,41.00


In [24]:
df["exact_match"].sum() / len(df)

np.float64(0.25)

In [153]:
# macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=metrics) * 100
macro_averaging(df[df["stage"] == "pre-edit"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacy_q0multi_hop_efficacy_q0mult...,2.2,49.709308,0.0,31.329471,7.311056,17.68
1,single_hop_efficacy_q0single_hop_efficacy_q0si...,5.9,56.474287,0.0,32.867520,11.050714,20.63
2,single_hop_efficacy_q1single_hop_efficacy_q1si...,6.1,55.455765,0.0,35.371379,12.870054,22.96


In [190]:
len(df[df["stage"] == "pre-edit"])

0

In [21]:
len(df[df["stage"] == "post-edit"])

9000

In [17]:
# df[(df["stage"] == "post-edit") & (df["question_type"] == "single_hop_efficacy")].sample(10)

In [28]:
(macro_averaging(df[df["stage"] == "post-edit"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["rouge1", "llm_accuracy"]) * 100).round(1) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM" question_tag

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacyq0multi_hop_efficacyq0multi_...,3.3,53.5,15.8,28.5
1,multi_hop_specificityq0multi_hop_specificityq0...,35.9,73.7,46.5,57.0
2,multi_hop_specificityq1multi_hop_specificityq1...,41.9,76.7,52.1,62.4
3,single_hop_efficacyq0single_hop_efficacyq0sing...,13.7,62.3,30.7,46.3
4,single_hop_efficacyq1single_hop_efficacyq1sing...,8.7,60.6,22.8,35.9
5,single_hop_specificityq0single_hop_specificity...,80.7,92.9,88.3,91.8
6,single_hop_specificityq1single_hop_specificity...,59.2,85.1,73.4,79.9
7,single_hop_specificityq2single_hop_specificity...,81.3,93.3,89.8,93.4
8,single_hop_specificityq3single_hop_specificity...,59.8,85.2,74.4,82.6


In [23]:
(macro_averaging(df[df["stage"] == "post-edit"], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["rouge1", "llm_accuracy"]) * 100).round(1) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM" question_tag

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacymulti_hop_efficacymulti_hop_...,7.7,64.3,16.4,24.8
1,multi_hop_specificitymulti_hop_specificitymult...,38.1,75.6,48.5,58.9
2,single_hop_efficacysingle_hop_efficacysingle_h...,23.8,73.0,33.9,40.7
3,single_hop_specificitysingle_hop_specificitysi...,69.9,89.6,80.4,85.5


In [25]:
(macro_averaging(df[df["stage"] == "pre-edit"], multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["rouge1", "llm_accuracy"]) * 100).round(1) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,rouge1,llm_accuracy
0,multi_hop_specificitymulti_hop_specificitymult...,40.6,76.4,50.4,61.4
1,single_hop_specificitysingle_hop_specificitysi...,73.3,90.6,82.3,87.5


In [20]:
df["stage"] == "pre-edit"

0       True
1       True
2       True
3       True
4       True
        ... 
5995    True
5996    True
5997    True
5998    True
5999    True
Name: stage, Length: 6000, dtype: bool

In [184]:
for q_tag, q_df in df[df["stage"] == "pre-edit"].groupby("question_tag"):
    print(q_tag + ":", sum(isinstance(x, str) for x in q_df["predicted_answer"].to_list()))

multi_hop_efficacy_q0: 999
single_hop_efficacy_q0: 802
single_hop_efficacy_q1: 744


In [45]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [11]:
save_dir = "/data/users/zliu/mend/debug_exp_output/Llama-3.2-1B-common-date-eos-sft_clm-baseline_lr=1e-05_epoch=4.0/individual_results"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

10100

In [13]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [14]:
all_df.head(2)

,id,question_type,question_tag,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
0,54,efficacy,efficacyq0,post-edit,When was the year after the year that Brian Mu...,2013,0,2005,0,1.0,8.0,8.0,0,0.50,0,0.235294
1,54,specificity,specificityq0,post-edit,When was the year after the year that Vincent ...,1854,0,1856,0,1.0,2.0,-2.0,0,0.75,0,0.368421


In [20]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [32]:
for i, row in nan_efficacy_df.iterrows():
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Charlotte Edwards was born?
1979 in Berlin. He started the career of Marketing Manager in 1995. In

When was the year after the year that Eric Stewart was born?
1932 in Buenos Aires

When was the year after the year that Emma Rogers was born?
1928 in Athens. 1948 she passed away.

When was the year after the year that Christina Gomez was born?
1930 in Berlin. She started the career of Chef in 1949. In 198

When was the year after the year that Laura Brown was born?
1967 in Istanbul. 1987 in Istanbul. 2002 in Istanbul. 200

When was the year after the year that Tyler Clark was born?
1904 in Moscow

When was the year after the year that Jennifer Roberts was born?
1940 in Jerusalem

When was th

In [34]:
born_question = 0
for i, row in efficacy_df.iterrows():
    born_question += "born" in row["question"]
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Brian Murphy passed away?
2005

When was the year after the year that Zoe Adams started the career of Paralegal?
1974

When was the year after the year that Victoria Phillips passed away?
2010

When was the year after the year that Madison Kelly passed away?
2003

When was the year after the year that John Reyes passed away?
1990

When was the year after the year that Eric Martinez was born?
1916

When was the year after the year that Aaron Morales started the career of Photographer?
1970

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Lucas Phillips passed away?
1959

When was the year after the year that Sofia Murphy started the career of Paralegal?
1906

When was the year after the year that Charl

In [35]:
born_question

28